In [69]:
import numpy as np
import pandas as pd
import sklearn.datasets
from sklearn.model_selection import train_test_split

In [70]:
dataset=sklearn.datasets.load_breast_cancer()
data=pd.DataFrame(dataset.data,columns=dataset.feature_names)

In [71]:
X=data.to_numpy()
data_label=pd.DataFrame(dataset.target)
Y=data_label.to_numpy()
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.20,random_state=42)

In [72]:
def sig(x):
    x = np.clip(x, -500, 500)
    return 1/(1+np.exp(-x))
def relu(x):
    return np.maximum(0, x)
def derrelu(x):
    return np.where(x > 0, 1, 0)

In [73]:
def initializeparameters(n0,n1,n2):
    W1=np.random.random((n1,n0))
    B1 = np.zeros((n1, 1))
    W2=np.random.random((n2,n1))
    B2 = np.zeros((n2, 1))
    parameters={
        'W1' :W1,
        'B1' :B1,
        'W2' :W2,
        'B2' :B2
    }
    return parameters
def forwardpropogation(X,parameters):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    Z1=(np.dot(W1,X.T)+B1).T
    A1=relu(Z1)
    Z2=(np.dot(W2,A1.T)+B2).T
    A2=sig(Z2)
    A2 = np.clip(A2, 1e-10, 1 - 1e-10)
    A2 = np.nan_to_num(A2, nan=1e-10)
    forward_cache={
        'Z1' :Z1,
        'A1' :A1,
        'Z2' :Z2,
        'A2' :A2

    }
    return forward_cache
def costfunction(Y,A2):
    m=Y.shape[0]
    cost = -(1/m)*np.sum(Y*np.log(A2)+(1-Y)*np.log(1-A2))
    return cost
def backwardpropagation(X,Y,parameters,forward_cache):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    Z1=forward_cache['Z1']
    A1=forward_cache['A1']
    Z2=forward_cache['Z2']
    A2=forward_cache['A2']
    m=Y.shape[0]
    dZ2=A2-Y
    dW2=(1/m)*np.dot(dZ2.T,A1)
    dB2=(1/m)*np.sum(dZ2,axis=0,keepdims=True).T
    dA1=np.dot(dZ2,W2)
    dZ1=dA1*derrelu(Z1)
    dW1=(1/m)*np.dot(dZ1.T,X)
    dB1=(1/m)*np.sum(dZ1,axis=0,keepdims=True).T
    gradients={
        'dW1' :dW1,
        'dB1' :dB1,
        'dW2' :dW2,
        'dB2' :dB2
    }
    return gradients
def updateparameters(parameters,gradients,learningrate):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    dW1=gradients['dW1']
    dW2=gradients['dW2']
    dB1=gradients['dB1']
    dB2=gradients['dB2']
    W1-=learningrate*dW1
    B1-=learningrate*dB1
    W2-=learningrate*dW2
    B2-=learningrate*dB2
    parameters={
        'W1' :W1,
        'B1' :B1,
        'W2' :W2,
        'B2' :B2
    }
    return parameters
def model(X,Y,n1,learningrate,iterations):
    n0=X.shape[1]
    n2=Y.shape[1]
    costlist=[]
    parameters=initializeparameters(n0,n1,n2)
    for i in range(iterations):
        forward_cache=forwardpropogation(X,parameters)
        cost=costfunction(Y,forward_cache['A2'])
        gradients=backwardpropagation(X,Y,parameters,forward_cache)
        parameters=updateparameters(parameters,gradients,learningrate)
        if(i%(iterations/10)==0):
            costlist.append(cost)
            print('cost after ',i,'th iterations is ',cost)
    return parameters,costlist

In [74]:
parameters,costlist=model(X_train,Y_train,1000,0.05,100)

cost after  0 th iterations is  8.552458886165748
cost after  10 th iterations is  0.6857851611897147
cost after  20 th iterations is  0.6800674801779687
cost after  30 th iterations is  0.6756179380546236
cost after  40 th iterations is  0.6721524823724171
cost after  50 th iterations is  0.669450918866742
cost after  60 th iterations is  0.6673427298031416
cost after  70 th iterations is  0.6656958881815364
cost after  80 th iterations is  0.6644081257686386
cost after  90 th iterations is  0.663400161031617


In [75]:
def accuracy(X,Y,parameters):
    forward_cache=forwardpropogation(X,parameters)
    A2=forward_cache['A2']
    for i in range(0,A2.shape[0]):
        if A2[i]<0.5:
            A2[i]=0
        else:
            A2[i]=1
    correct=0
    for i in range(0,A2.shape[0]):
        if(A2[i]==Y[i]):
            correct+=1
    return (correct/A2.shape[0])*100

print('The accuracy of the model on training set is ',round(accuracy(X_train,Y_train,parameters),2),'%')
print('The accuracy of the model on testing set is ',round(accuracy(X_test,Y_test,parameters),2),'%')

The accuracy of the model on training set is  62.86 %
The accuracy of the model on testing set is  62.28 %
